In [1]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random
from tqdm import tqdm

torch.backends.cudnn.deterministic = True

## General Settings

In [2]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-5
BATCH_SIZE = 16
NUM_EPOCHS = 4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

PATH = 'best_model.pth' # PATH to save and load model

MAX_LEN=64

cuda


## Dataset

In [3]:
TEXT = data.Field(tokenize='spacy',
                  include_lengths=True,
                 fix_length=MAX_LEN)
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [4]:
TEXT.build_vocab(train_data,
                 max_size=VOCABULARY_SIZE,
                 vectors='glove.6B.100d',
                 unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [5]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=DEVICE)

In [6]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([64, 16])
Target vector size: torch.Size([16])

Valid:
Text matrix size: torch.Size([64, 16])
Target vector size: torch.Size([16])

Test:
Text matrix size: torch.Size([64, 16])
Target vector size: torch.Size([16])


## model

In [7]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
#     mirror='tuna',
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [8]:
optimizer = AdamW(model.parameters(),
                  lr = LEARNING_RATE, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [9]:
def train_val(model,optimizer,criterion,train_loader, valid_loader, epochs, path):
    since = time.time()
    min_val_loss = 1e10
    for epoch in range(epochs):
        train_loss=0.0
        val_loss=0.0
        val_acc = 0.0
        
        # trainning
        model.train()
        print('training...')
        for idx, batch in enumerate(tqdm(train_loader)):
            if idx%50== 0 and not idx == 0:
                print("batch:",idx,"loss:",loss.data.item())
            optimizer.zero_grad()
            inputs, labels=batch.text, batch.label
#             print(inputs[0].size())
            # print(inputs[1].size())
            inputs=inputs[0].transpose(0,1).to(DEVICE)
            labels=labels.to(torch.int64).to(DEVICE)
            outputs = model(inputs,token_type_ids=None,labels=labels)
#             print(labels)
            loss = outputs[0]
        
            train_loss += loss.data.item()
            
#             Before=list(model.parameters())[0].clone()
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
#             After = list(model.parameters())[0].clone()
            
#             print("update:",torch.sum(After-Before))
            
            # print(batch.text[0].size(0))
        train_loss /= len(train_loader)
        # print(len(train_loader))
        print(f"epoch: {epoch+1}: train loss:{train_loss}")
        
        print("valing")
        model.eval()
        with torch.no_grad():
            for val_idx, val_batch in enumerate(tqdm(valid_loader)):
                val_inputs, val_labels=val_batch.text, val_batch.label
                val_inputs = val_inputs[0].transpose(0,1).to(DEVICE)
                val_labels = val_labels.to(torch.int64).to(DEVICE)
                val_outputs =  model(val_inputs,token_type_ids=None)[0]
                val_outputs_acc = torch.argmax(val_outputs,axis=1).view(-1)
                val_acc+=torch.sum(val_outputs_acc==val_labels)
                loss = criterion(val_outputs,val_labels)
                val_loss += loss.data.item()
        val_loss /= len(valid_loader)
        val_acc /= len(valid_data)
        print(f"epoch: {epoch+1}: val loss:{val_loss} val acc:{val_acc}")

#         if val_loss<min_val_loss:
#             min_val_loss = val_loss
#             torch.save(model.state_dict(),path)
#             print('model saved!')

        
    time_elapsed = time.time() - since
    print('\nTraining complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [10]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
train_val(model,optimizer,criterion,train_loader, valid_loader, NUM_EPOCHS, PATH)


training...


  4%|███▏                                                                            | 50/1250 [01:38<11:49,  1.69it/s]

batch: 50 loss: 0.6383523941040039


  8%|██████▍                                                                        | 101/1250 [02:09<11:33,  1.66it/s]

batch: 100 loss: 0.6873492002487183


...

batch: 1200 loss: 0.565887987613678


100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [12:34<00:00,  1.66it/s]


epoch: 4: train loss:0.5124421606302262
valing


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:53<00:00,  5.90it/s]

epoch: 4: val loss:0.5488730688064624 val acc:0.7319999933242798

Training complete in 55m 7s


In [11]:
def test(model, test_loader):
    acc = 0.0
    test_loss = 0.0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(test_loader)):
            inputs, labels=batch.text, batch.label
            inputs = inputs[0].transpose(0,1).to(DEVICE)
            labels = labels.to(torch.int64).to(DEVICE)
            outputs = model(inputs,token_type_ids=None)[0]
            loss = criterion(outputs,labels)
            test_loss += loss.data.item()
            outputs = torch.argmax(outputs,axis=1).view(-1)
            acc+=torch.sum(outputs==labels)
    acc /= len(test_data)
    test_loss /= len(test_loader)
    print(f"test loss: {test_loss}: test acc:{acc}")

In [12]:
test(model, test_loader)

100%|██████████████████████████████████████████████████████████████████████████████| 1563/1563 [04:46<00:00,  5.46it/s]

test loss: 0.5694077742534498: test acc:0.7275999784469604
